## File for fitting EDES to all the sets generated for MIR, LIR and healthy populations

In [ ]:
using CSV # for reading the data
using DataFrames # for storing the data


include("../EDES/including.jl") # for the EDES model 